In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import import_ipynb
#import Segmentation
import time
import preds

importing Jupyter notebook from preds.ipynb
importing Jupyter notebook from Captcha_seg.ipynb
1080


/home/bg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 864 samples, validate on 216 samples
Epoch 1/20
864/864 [==============================] - 7s 8ms/step - loss: 3.5887 - acc: 0.0255 - val_loss: 3.5859 - val_acc: 0.0324
Epoch 2/20
864/864 [==============================] - 6s 7ms/step - loss: 3.4138 - acc: 0.0718 - val_loss: 3.1149 - val_acc: 0.1481
Epoch 3/20
864/864 [==============================] - 7s 8ms/step - loss: 2.4277 - acc: 0.2963 - val_loss: 2.4283 - val_acc: 0.3426
Epoch 4/20
864/864 [==============================] - 8s 9ms/step - loss: 1.5138 - acc: 0.5532 - val_loss: 1.7494 - val_acc: 0.5231
Epoch 5/20
864/864 [==============================] - 7s 9ms/step - loss: 0.9571 - acc: 0.7072 - val_loss: 1.5150 - val_acc: 0.5231
Epoch 6/20
864/864 [==============================] - 8s 9ms/step - loss: 0.6320 - acc: 0.8171 - val_loss: 1.4403 - val_acc: 0.6157
Epoch 7/20
864/864 [==============================] - 10s 11ms/step - loss: 0.3953 - acc: 0.8727 - val_loss: 1.5202 - val_acc: 0.5833
Epoch 8/20
864/864 [========

In [2]:
CATEGORIES = set()
names =[]
for i in os.listdir('test/'):
    names.append(i.split('.')[0])
for i in names:
    for x in list(i):
        CATEGORIES.add(x)
CATEGORIES=list(CATEGORIES)
print(CATEGORIES)

['e', 'B', '%', 'R', '&', 'H', 'A', 'T', '4', 'J', 'b', 'N', 'I', 'E', 'Y', 'P', 'M', 'r', '3', 'd', 'K', 'n', 'a', '6', 'W', 'O', 'Q', '5', '@', 'X', 'C', 'V']


In [3]:
#print(len(train_data))

In [4]:
import random

#random.shuffle(train_data)

In [5]:
#for sample in train_data[:10]:
#    print(sample[1])

In [6]:
X = []
y = []

for features, label in train_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in= open("X.pickle", "rb")
X = pickle.load(pickle_in)
X.shape[1:]

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0

model = Sequential()

model.add(Conv2D(64,(3,3),input_shape= X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))


model.add(Dense(19))
model.add(Activation("softmax"))

from keras.utils import to_categorical
y_binary = to_categorical(y)

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X,y_binary,batch_size=128, epochs=20, validation_split = 0.2)


In [7]:
#loss_value, metrics_value = model.evaluate(X,y_binary,batch_size=10)

In [8]:
#print(loss_value, metrics_value)

In [9]:
import random

In [10]:
def similarity(a,b):
    i =0
    for x,y in zip(a,b):
        if x==y:
            i+= 1
    percentage = (i/5) * 100
    return percentage

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
    #plt.imshow(new_array,'gray')
    return new_array.reshape(-1, IMG_SIZE,IMG_SIZE,1)

total_time = 0
score = 0
count = 0
done = []
list_ = preds.predictions('test',CATEGORIES)
CAPTCHA_DIR = list_[0]
for x in CAPTCHA_DIR:
    captchas = random.choice(CAPTCHA_DIR)
    if captchas in done:
        continue
    else:
        done.append(captchas)
        start_time = time.time()
        predict_list= []
        print('\n')
        print('Actual: ',captchas[0][0])

        files = []
        for segments in list_:
            files.append(segments)

        #for segments in files:
        #    filepath = (SEGMENTS_DIR,segments)
        #    prediction = model.predict([prepare(filepath)])
        #    predict_list.append(CATEGORIES[np.argmax(prediction[0])])

        elapsed_time = time.time()-start_time
        time.sleep(captchas[1])
        total_time += elapsed_time
        final_prediction = ''.join(predict_list)
        #if captcha_name[0]== final_prediction:
        #    score +=1
        score = list_[1]
        print('Predicted:',captchas[0][1],'\n')
        print('Elapsed time= ',captchas[1],'s')
        total_time +=captchas[1]
        print('<------------------------------->')    
    

print('Total number of CAPTCHAS broken: ',score)    
print('Total number of CAPTCHAS attempted: ',len(list_[0]))
print('Total time taken= ',total_time,'s')
print('Accuracy= ', (score/len(list_[0])*100))
#model = tf.keras.models.load_model('captcha_alpha.model')
#prediction = model.predict([prepare('3.png')])
#print(len(prediction[0]))
#print(prediction[0])
#print(CATEGORIES[np.argmax(prediction[0])])



Actual:  &6NnEN
Predicted: &6NnEN 

Elapsed time=  0.7500156909892751 s
<------------------------------->


Actual:  OTTaRTb
Predicted: OTTaRTb 

Elapsed time=  0.6564908060422623 s
<------------------------------->


Actual:  TNHa
Predicted: TNHa 

Elapsed time=  0.2879147194352575 s
<------------------------------->


Actual:  CNMA
Predicted: 3Xr@ 

Elapsed time=  0.8828896436592134 s
<------------------------------->


Actual:  naHa4eR
Predicted: b6aE 

Elapsed time=  0.07012869738615912 s
<------------------------------->


Actual:  XnA@OH
Predicted: XnA@OH 

Elapsed time=  0.8113209873564968 s
<------------------------------->


Actual:  MOTbn
Predicted: IaHT 

Elapsed time=  0.8974222531509897 s
<------------------------------->


Actual:  rPNnb
Predicted: &QNn 

Elapsed time=  0.8875022973244159 s
<------------------------------->


Actual:  nOTNP
Predicted: H&Ja 

Elapsed time=  0.25275157322495767 s
<------------------------------->


Actual:  6YnOHb
Predicted: 6YnOHb 

Elap

Predicted: 6aWMaK 

Elapsed time=  0.7412133730373655 s
<------------------------------->


Actual:  @O%aP
Predicted: @O%aP 

Elapsed time=  0.1169089353597369 s
<------------------------------->


Actual:  POrYH
Predicted: PXbb 

Elapsed time=  0.8806337295362451 s
<------------------------------->


Actual:  OT6ONKa
Predicted: %dbV 

Elapsed time=  0.4846419213507249 s
<------------------------------->


Actual:  nOrNH
Predicted: rNKO 

Elapsed time=  0.31742539121773483 s
<------------------------------->


Actual:  nOnn34P
Predicted: nOnn34P 

Elapsed time=  0.9429473931731333 s
<------------------------------->


Actual:  O6On
Predicted: VIbJ 

Elapsed time=  0.7666851092816709 s
<------------------------------->


Actual:  6NeHNe
Predicted: 6NeHNe 

Elapsed time=  0.048976696326010205 s
<------------------------------->


Actual:  PaNnO
Predicted: PaNnO 

Elapsed time=  0.03459155844604256 s
<------------------------------->


Actual:  O6On
Predicted: O6On 

Elapsed time=  0.0134

Predicted: YQnP 

Elapsed time=  0.34749919576949584 s
<------------------------------->


Actual:  ra6aPNT
Predicted: ra6aPNT 

Elapsed time=  0.8324232926078804 s
<------------------------------->


Actual:  %nNrenb
Predicted: Eaan 

Elapsed time=  0.33031314371313336 s
<------------------------------->


Actual:  BAHHEC
Predicted: 5R3B 

Elapsed time=  0.512251501144461 s
<------------------------------->


Actual:  EnEN
Predicted: EnEN 

Elapsed time=  0.6473833734570893 s
<------------------------------->


Actual:  XEPEC
Predicted: XEPEC 

Elapsed time=  0.45309214617705107 s
<------------------------------->


Actual:  XnOPN4
Predicted: XnOPN4 

Elapsed time=  0.7906038308509328 s
<------------------------------->


Actual:  AO%aP
Predicted: AO%aP 

Elapsed time=  0.18798803747295534 s
<------------------------------->


Actual:  B3PE3
Predicted: B3PE3 

Elapsed time=  0.6673634711234517 s
<------------------------------->


Actual:  Xn&nNK
Predicted: Xn&nNK 

Elapsed time=  0.5

Predicted: R%a& 

Elapsed time=  0.2378461948306858 s
<------------------------------->


Actual:  6OTNKN
Predicted: 6OTNKN 

Elapsed time=  0.9217449639892443 s
<------------------------------->


Actual:  %3HNn
Predicted: anJr 

Elapsed time=  0.8279972872396789 s
<------------------------------->


Actual:  3nb%
Predicted: @&5% 

Elapsed time=  0.6516305459787157 s
<------------------------------->


Actual:  XEPEC
Predicted: XEPEC 

Elapsed time=  0.28396929281377425 s
<------------------------------->


Actual:  @Onb4E
Predicted: 3dB6 

Elapsed time=  0.3600536298465302 s
<------------------------------->


Actual:  OPaHMa@
Predicted: ROO@ 

Elapsed time=  0.7352411959002485 s
<------------------------------->


Actual:  CNMA
Predicted: CNMA 

Elapsed time=  0.6613105292964135 s
<------------------------------->


Actual:  3eMnR
Predicted: A6eA 

Elapsed time=  0.8899621800826226 s
<------------------------------->


Actual:  KaCNMOB
Predicted: AJ6b 

Elapsed time=  0.840902955523